This notebook allows to quickly plot new monthly data from ongoing runs

In [ ]:
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import copy
import sys

sys.path.append('../src')

from rundata import RunData

#np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
#runs = ['ctrl','half','hal2','totl','tot2']
runs = ['ctrl','tot2','amun']

In [ ]:
for run in runs:
    rd = RunData(run)
    rd.checkfornewdata(option='mon')

In [ ]:
plt.style.use('../src/style_paper')

mpl.rc('figure',figsize=(16,8))

In [ ]:
tmax = 100*12
nbasins = 5 #5 or 7

fig,ax = plt.subplots(3,nbasins,sharex=True,sharey='row')

ctrl = xr.open_dataset(f'../data/temperature_mon_ctrl.nc').isel(time=slice(0,tmax))

#Get linear fit through control
ctrl2 = ctrl.temp.copy()
for b,bas in enumerate(ctrl.basin.values):
    out = np.polyfit(ctrl.time,ctrl.temp.sel(basin=bas),1)
    ctrl2[:,b] = out[1] + ctrl.time*out[0]


for r,run in enumerate(runs):
    ds = xr.open_dataset(f'../data/temperature_mon_{run}.nc').isel(time=slice(0,tmax))
    for b,bas in enumerate(ds.basin.values[:nbasins]):
        ax[0,b].plot(ds.time/12,ds.temp.sel(basin=bas),c=rd.rcol[run],label=run)
        ax[1,b].plot(ds.time/12,ds.temp.sel(basin=bas)-ctrl.temp.sel(basin=bas),c=rd.rcol[run])
        if run != 'ctrl':
            ax[2,b].plot(ds.time/12,ds.temp.sel(basin=bas)-ctrl2.sel(basin=bas),c=rd.rcol[run])
        else:
            ax[0,b].plot(ds.time/12,ctrl2.sel(basin=bas),c=rd.rcol[run],lw=.5)
        ax[2,b].axhline(0,0,1,c=rd.rcol['ctrl'],lw=.5)
        ax[0,b].set_title(bas,color=rd.bcol[bas],loc='left')
    ds.close()
    
ax[0,0].legend()
ax[0,0].set_ylabel('Subsurface Temperature [degC]')
ax[1,0].set_ylabel('Temp anom from ctrl [degC]')
ax[2,0].set_ylabel('Temp anom from fit through ctrl [degC]')
fig.supxlabel('Time [years]')

plt.savefig(f'../draftfigs/dumfig_perturbation.png',dpi=450,facecolor='w',transparent=False)
plt.show()